In [16]:
import pandas as pd
from gensim.models import Word2Vec
import networkx as nx

In [3]:
import sys
sys.path.append('/workspaces/master_thesis/poincare')
from build_graph import Builder

In [4]:
relations=Builder('/workspaces/master_thesis/CONCEPT_RELATIONSHIP.csv', '/workspaces/master_thesis/CONCEPT.csv')
relations=list(relations())

/workspaces/master_thesis/poincare/build_graph.py:11: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data_concept = pd.read_csv(path_concept, on_bad_lines='skip', sep='\t')
/workspaces/master_thesis/poincare/build_graph.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_merge_2=data_merge_2[data_merge_2.standard_concept_x=='S'][data_merge_2.standard_concept_y=='S']


In [7]:
relations[0]

(281, 280)

In [11]:
#take only part of relations
relations=relations[:10000]

In [12]:
G = nx.Graph()
G.add_edges_from(relations)

In [13]:
import random

In [14]:
def get_random_walk(graph:nx.Graph, node:int, n_steps:int = 20):
   """ Given a graph and a node, 
       return a random walk starting from the node 
   """
   local_path = [str(node),]
   target_node = node
   for _ in range(n_steps):
      neighbors = list(nx.all_neighbors(graph, target_node))
      target_node = random.choice(neighbors)
      local_path.append(str(target_node))
   return local_path


walk_paths = []
for node in G.nodes():
   for _ in range(10):
      walk_paths.append(get_random_walk(G, node))

In [15]:
walk_paths[0]

['281',
 '362',
 '281',
 '280',
 '364',
 '280',
 '365',
 '366',
 '365',
 '280',
 '365',
 '280',
 '281',
 '280',
 '285',
 '308',
 '285',
 '363',
 '285',
 '286',
 '285']

In [17]:
# Instantiate word2vec model
embedder = Word2Vec(
   window=20, sg=1, hs=0, negative=10, alpha=0.03, min_alpha=0.0001,    
   seed=42
)
# Build Vocabulary
embedder.build_vocab(walk_paths, progress_per=2)
# Train
embedder.train(
   walk_paths, total_examples=embedder.corpus_count, epochs=20, 
   report_delay=1
)

(35737800, 35737800)

In [18]:
embedder.wv.most_similar('281')

[('282', 0.9989911913871765),
 ('284', 0.9988034963607788),
 ('326', 0.9984772801399231),
 ('280', 0.9984484910964966),
 ('283', 0.9984161257743835),
 ('348', 0.9981974959373474),
 ('362', 0.9980472326278687),
 ('292', 0.9979079961776733),
 ('293', 0.9969819784164429),
 ('409', 0.9969725012779236)]